In [1]:
import wandb
from pathlib import Path
import os
from tqdm.auto import tqdm
from PIL import Image
import numpy as np
import torch
from torch import nn
from datasets import load_dataset
from transformers import MaskFormerImageProcessor, MaskFormerForInstanceSegmentation
from torch.utils.data import Dataset

In [5]:
os.environ['WANDB_NOTEBOOK_NAME']='EDA.ipynb'
os.environ["KMP_DUPLICATE_LIB_OK"]='TRUE'

In [6]:
Logging_Place= 'LOCAL'

if Logging_Place == 'LOCAL':
    os.environ['WANDB_BASE_URL']="http://10.24.1.19:8080"
    os.environ['WANDB_API_KEY']='local-96dd72cbb60b0155149e2f9dc985a636ffa77b28'
    ! wandb login --host=http://10.24.1.19:8080

elif Logging_Place =="CLOUD":
    os.environ['WANDB_BASE_URL']="https://api.wandb.ai"
    os.environ['WANDB_API_KEY']='be69ec2b537cb972b0106fabd867f0dc68c4468a'
    ! wandb login --host=https://api.wandb.ai

^C


wandb: Network error (HTTPError), entering retry loop.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [ ]:
data_path=Path('COD10K-v2')
train_path=data_path / 'Train'
test_path= data_path / 'Test'

train_images_path= train_path / 'Images/Image'
train_labels_path= train_path / 'GT_Objects/GT_Object'

In [ ]:
train_images_list=list(train_images_path.glob('*.jpg'))
print(train_images_list[0:5])
train_labels_list=list(train_labels_path.glob('*.png'))
print(train_labels_list[0:5])

In [ ]:
super_classes_list=[ i.stem.split('-')[3] for i in train_images_list]
super_classes_list=[*set(super_classes_list)]
print(super_classes_list)


In [ ]:
index=5
sub_classes_list=[]
for i in train_images_list:
    split_path=i.stem.split('-')
    count=len(split_path)
    if (index<count):
        sub_classes_list.append(split_path[index])
sub_classes_list=[*set(sub_classes_list)]
print(sub_classes_list)

#####    Setting up dataloaders

In [12]:
model = MaskFormerForInstanceSegmentation.from_pretrained("maskformer-swin-base-ade")

In [13]:
weights=MaskFormerForInstanceSegmentation.

MaskFormerForInstanceSegmentation(
  (model): MaskFormerModel(
    (pixel_level_module): MaskFormerPixelLevelModule(
      (encoder): MaskFormerSwinBackbone(
        (model): MaskFormerSwinModel(
          (embeddings): MaskFormerSwinEmbeddings(
            (patch_embeddings): MaskFormerSwinPatchEmbeddings(
              (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
            )
            (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (encoder): MaskFormerSwinEncoder(
            (layers): ModuleList(
              (0): MaskFormerSwinStage(
                (blocks): ModuleList(
                  (0): MaskFormerSwinLayer(
                    (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
                    (attention): MaskFormerSwinAttention(
                      (self): MaskFormerSwinSelfAttention(
                        (query): Linear(in_featur

In [ ]:
class CloudDataset(Dataset):
    def __init__(
        self,
        df: pd.DataFrame = None,
        datatype: str = "train",
        img_ids: np.array = None,
        transforms=albu.Compose([albu.HorizontalFlip()]), #, AT.ToTensor()
    ):
        self.df = df
        if datatype != "test":
            self.data_folder = f"{img_paths}/train_images_525/train_images_525"
        else:
            self.data_folder = f"{img_paths}/test_images_525/test_images_525"
        self.img_ids = img_ids
        self.transforms = transforms

    def __getitem__(self, idx):
        image_name = self.img_ids[idx]
        mask = make_mask(self.df, image_name)
        image_path = os.path.join(self.data_folder, image_name)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        augmented = self.transforms(image=img, mask=mask)
        img = np.transpose(augmented["image"], [2, 0, 1])
        mask = np.transpose(augmented["mask"], [2, 0, 1])
        return img, mask

    def __len__(self):
        return len(self.img_ids)